In [1]:
import torch
import numpy as np
import sys
sys.path.append('/burg/home/jb5005')
from struct_diff.data.datamodule import ContinuousStructTokenDataModule
from struct_diff.data.dataset import ContinuousStructTokenDataset
from struct_diff.model.models import TransformerModel
from esm.pretrained import ESM3_structure_encoder_v0, ESM3_structure_decoder_v0
import time

from struct_diff.model.layers import TransformerLayer, positional_encoding, time_encoding

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

In [2]:
dataset.__getitem__(183).shape

NameError: name 'dataset' is not defined

In [3]:
dataset = ContinuousStructTokenDataset(base_path = '/pmglocal/jb5005/struct_diff_data/data', 
                                       df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv')

for i in range(2000):
    thingy = dataset.__getitem__(i)
    try:
        if thingy.shape[1] != 1280:
            print(i)
    except:
        print('error')
        print(i)
    if i == 1999:
        print('done')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


done


In [4]:
import sys
sys.path.append('/burg/home/jb5005')
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor
import matplotlib.pyplot as plt
import torch
import wandb

from struct_diff.data.flow import interpolate
from struct_diff.data.datamodule import ContinuousStructTokenDataModule
from struct_diff.model.models import TransformerModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_transformer_heads = 8
architecture = "transformer"
batching = "padding"

wandb_logger = WandbLogger(project="struct_diff_v0")
wandb_logger.experiment.config["batching"] = batching
wandb_logger.experiment.config["architecture"] = architecture
wandb_logger.experiment.config["nhead"] = num_transformer_heads


#Load the data
datamodule = ContinuousStructTokenDataModule(base_path = '/pmglocal/jb5005/struct_diff_data/data', 
                                             df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv',
                                             num_workers = 0,
                                             decoder = None)
datamodule.setup('train')

#Setup model
model = TransformerModel()
model = model.to(device)

trainer = pl.Trainer(max_epochs=20, enable_progress_bar=True, logger = wandb_logger)

trainer.fit(model = model, datamodule= datamodule)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: johndboom. Use `wandb login --relogin` to force relogin


/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/pmglo

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [2]:
datamodule = ContinuousStructTokenDataModule(base_path = '/pmglocal/jb5005/struct_diff_data/data', 
                                             df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv',
                                             num_workers = 0,
                                             decoder = None)
datamodule.setup('train')

dataloader = datamodule.train_dataloader()
first_element = next(iter(dataloader))

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/burg/

In [5]:
model = TransformerModel()
model = model.to(device)

model(first_element, torch.tensor(0.5, device = "cuda"))

tensor([[[ 0.7534,  0.9418,  0.3182,  ...,  1.6682, -0.3786,  0.3969],
         [ 0.8566,  0.4398,  0.2601,  ...,  1.3953, -1.6668,  0.3695],
         [ 0.9763,  0.0325,  0.0487,  ...,  1.6311, -0.3386,  0.4285],
         ...,
         [ 0.9666, -0.6716,  0.7412,  ...,  1.8030, -0.4208,  1.9236],
         [ 1.0688, -1.5287, -0.0336,  ...,  1.7772, -0.4034,  1.9060],
         [ 0.4166, -2.0776, -0.1557,  ...,  1.7663, -0.4316,  1.8877]],

        [[ 3.0384,  0.2797, -0.1461,  ...,  0.1311, -2.3067, -0.7793],
         [-0.2728, -0.1264,  1.4604,  ...,  2.0726, -0.4957, -0.3555],
         [ 1.7539,  0.3666,  1.4186,  ...,  1.1522, -0.2600,  0.1991],
         ...,
         [ 0.8487, -0.7989,  0.7671,  ...,  1.5496, -0.3610,  1.8884],
         [ 0.9543, -1.6730, -0.0278,  ...,  1.5306, -0.3484,  1.8791],
         [ 0.2843, -2.2296, -0.1536,  ...,  1.5266, -0.3815,  1.8707]],

        [[ 1.5106,  1.8138, -0.0817,  ...,  2.4542, -0.8729,  1.2898],
         [ 1.4226,  0.6056,  0.4753,  ...,  1

In [8]:
first_element

tensor([[[-0.0143,  0.0996,  0.0006,  ...,  0.0136,  0.0469, -0.0461],
         [-0.0137,  0.0693, -0.0767,  ...,  0.0923, -0.0786, -0.0820],
         [-0.1328,  0.0090, -0.1904,  ..., -0.0011,  0.0718,  0.0579],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0649,  0.0060, -0.1318,  ..., -0.0233, -0.0035,  0.0391],
         [-0.1006, -0.0466,  0.0201,  ..., -0.0097, -0.0304, -0.0025],
         [ 0.0131, -0.0364, -0.0104,  ..., -0.1143, -0.0396,  0.0276],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0371,  0.0266, -0.0046,  ...,  0.0262, -0.0684, -0.0359],
         [ 0.0513, -0.1279, -0.0012,  ..., -0

/burg/home/jb5005/struct_diff/data/dataset.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(cont_emb)


In [4]:
dataset = ContinuousStructTokenDataset(base_path = '/pmglocal/jb5005/struct_diff_data/data', df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv')

In [8]:
dataset.__getitem__(18).shape

Time to load pdb: 0.0007641315460205078
Time to create protein object: 5.0067901611328125e-06
Time to convert to continuous embeddings: 0.0003159046173095703


torch.Size([104, 1280])

In [17]:
decoder = ESM3_structure_decoder_v0(device='cuda')

In [21]:
start = time.time()
arr = np.arange(0,1000)
print(decoder.embed(torch.tensor(arr, device = "cuda")).shape)
print(f"Time taken for embedding: {time.time() - start}")

torch.Size([1000, 1280])
Time taken for embedding: 0.0008194446563720703


In [5]:
first_element.shape

torch.Size([32, 296, 1280])

In [ ]:
first_element[0,:,:]

tensor([[-0.0123,  0.0757, -0.0201,  ...,  0.0581, -0.0137, -0.0153],
        [ 0.0508,  0.1787, -0.0540,  ..., -0.0391,  0.0718,  0.0049],
        [ 0.0903, -0.0347,  0.0090,  ..., -0.0459, -0.0913,  0.0542],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

In [9]:
tensor_to_add = torch.randn([296,1280]).to('cuda')

result = first_element + tensor_to_add.unsqueeze(0)

In [ ]:
batch = torch.zeros([32,296,1280]).to('cuda')
result = batch + tensor_to_add.unsqueeze(0)


tensor([[[-2.6396,  2.3772, -0.3354,  ...,  1.4915,  2.4224,  2.4508],
         [ 0.0118,  0.8667,  0.4948,  ..., -1.0640,  0.3702, -1.0209],
         [-1.4208, -0.6164,  1.2664,  ...,  0.1258,  0.6534,  0.6582],
         ...,
         [ 1.0933, -0.7741, -0.1789,  ...,  0.2260,  0.6766,  0.1971],
         [-0.3510,  0.8482,  0.6945,  ...,  0.3624,  0.2226,  1.2152],
         [-0.9116, -0.5517,  1.9972,  ..., -0.7783,  1.4103, -0.7878]],

        [[-2.6396,  2.3772, -0.3354,  ...,  1.4915,  2.4224,  2.4508],
         [ 0.0118,  0.8667,  0.4948,  ..., -1.0640,  0.3702, -1.0209],
         [-1.4208, -0.6164,  1.2664,  ...,  0.1258,  0.6534,  0.6582],
         ...,
         [ 1.0933, -0.7741, -0.1789,  ...,  0.2260,  0.6766,  0.1971],
         [-0.3510,  0.8482,  0.6945,  ...,  0.3624,  0.2226,  1.2152],
         [-0.9116, -0.5517,  1.9972,  ..., -0.7783,  1.4103, -0.7878]],

        [[-2.6396,  2.3772, -0.3354,  ...,  1.4915,  2.4224,  2.4508],
         [ 0.0118,  0.8667,  0.4948,  ..., -1

In [15]:
mask = first_element == 0.0

In [17]:
mask[0,:,:]

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True]], device='cuda:0')